In [1]:
from lsst.ts import salobj
from lsst.ts.idl.enums import MTMount
import logging
import asyncio
import os
import yaml

import numpy as np
from matplotlib import pyplot as plt
from astropy.time import Time
from datetime import datetime, timedelta
import pandas as pd
import astropy.units as u

from lsst_efd_client import EfdClient

/opt/lsst/software/stack/conda/miniconda3-py38_4.9.2/envs/lsst-scipipe-0.4.1/lib/python3.8/site-packages/jose/backends/cryptography_backend.py:23: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes, int_to_bytes


# Before the tests:

In [2]:
script = salobj.Controller("Script", index=42658885)

In [3]:
rot_sim = salobj.Remote(script.domain, "MTRotator")
await rot_sim.start_task

In [ ]:
# this is to ensure that MTMount is connected and telemetry is send
rot_sim_connected = await rot_sim.evt_connected.aget(timeout=10.) # This is how you get an event/telemetry with await remote.type_name.aget() type being [evt,tel] and name being the name of the topic
rot_sim_commandable = rot_sim_connected.command
rot_sim_telemetry_working = rot_sim_connected.telemetry

if not rot_sim_connected or not rot_sim_telemetry_working:
    raise Exception("Mount not connected or telemetry not being received.")

In [4]:
#This is to check what the controller state is:
controller_state = await rot_sim.evt_controllerState.aget(timeout=10.)
state = controller_state.controllerState
#offline_substate = controller_state.offlineSubstate
print(state)

TimeoutError: 

In [5]:
#This is to check MTMount is commandable through SAL:
commandable_by_dds = await rot_sim.evt_commandableByDDS.aget(timeout=10.)
dds_state = commandable_by_dds.state
if not dds_state:
    raise Exception("Controller must in CommandableByDDS state.")

TimeoutError: 

In [9]:
#Print the available MTMount events

print("Telemery:", rot_sim.salinfo.telemetry_names)
print("Events:", rot_sim.salinfo.event_names)
print("Commands:", rot_sim.salinfo.command_names)

AttributeError: 'tuple' object has no attribute 'rotation'

In [10]:
#Send the MTMount simulator component into STANDBY
await salobj.set_summary_state(rot_sim, salobj.State.STANDBY)

RuntimeError: Cannot get summaryState from MTRotator

In [11]:
#Check if Rotator telemetry is getting into the EFD
#Add telemetry as needed
#For the "setCompensateMode" of the Hexapods the "angle" of the rotator is needed
rot_sim_Rotation_post_stop_1= await rot_sim.tel_rotation.aget(timeout=10.)
print("Simulated mount elevation", rot_sim_Rotation_post_stop_1.actualPosition)
print("Simulated mount elevation", rot_sim_Rotation_post_stop_1.timestamp)
await asyncio.sleep(10)

TimeoutError: 

In [8]:
#Check if Rotator telemetry has changed in the EFD
#Add telemetry as needed
rot_sim_Rotation_post_stop_2= await rot_sim.tel_rotation.aget(timeout=10.)
print("Simulated rotator angle", rot_sim_rotation_post_stop_2.timestamp)

TimeoutError: 

In [ ]:
#Check if the input to the EFD has stopped
diff_time = rot_sim_Rotator_post_stop_1.timestamp - rot_sim_Rotator_post_stop_2.timestamp
print (diff_time)

if (rot_sim_Rotator_post_stop_1.timestamp - rot_sim_Rotator_post_stop_2.timestamp==0):
    print("Rotator has stopped sending telemetry")
else:
    raise Exception("Rotator is still sending telemetry!")

In [ ]:
#Stop the script controller
await script.close()

In [12]:
#Start the MTMount controller
rot_control = salobj.Controller(name="MTRotator")
await rot_control.start_task

RuntimeError: Cannot find IDL file /opt/lsst/src/ts_idl/idl/sal_revCoded_Rotator.idl for name='Rotator'

In [13]:
#To check if the input to the EFD has started prepare the EFD readout
#may need to wait a few seconds before event shows up in EFD
client = EfdClient('ncsa_teststand_efd')
#client = EfdClient('summit_efd')

In [14]:
#Send a test command 
rot_control.evt_target.set_put(rotation=3)

NameError: name 'rot_control' is not defined

In [15]:
#Read from the EFD
end = Time(datetime.now())
start = end - timedelta(hours=3)
logeventTarget = await client.select_time_series('lsst.sal.MTRotator.logevent_target', '*', start.tai, end.tai)
print("Rotator controller rotation from the EFD:")
logeventTarget.position

Rotator controller rotation from the EFD:


AttributeError: 'DataFrame' object has no attribute 'position'

# Set the MTMount Controller to the needed values and exectute the tests from another notebook:

In [ ]:
# Set the elevation angle. Do this for every desired elevation angle.
rotator.evt_positon.set_put(position=1)

# After the tests:

In [ ]:
#Stop the MTMount controller
await rot_control.close()

In [ ]:
#Send a test command
rotator.evt_positon.set_put(position=1)

In [ ]:
logeventTarget = await client.select_time_series('lsst.sal.MTRotator.logevent_target', '*', start.tai, end.tai)
print("Rotator controller rotation from the EFD:")
logeventTarget.position

In [ ]:
#Check that no more telemetry from the MTMount controller is going into the EFD
rot_control_Elevation= await rot_contorl.tel_elevation.aget(timeout=10.)
print("Mount controller elevation", rot_control_Elevation.angleActual)

In [ ]:
#Start the script controller
script = salobj.Controller("Script", index=42658885)

In [ ]:
#Start the mount Simluator controller
await rot_sim.start_task

In [ ]:
#Send the MTMount simulator component into ENABLE again
await salobj.set_summary_state(rot, salobj.State.ENABLE)

In [ ]:
#Check that the EFD is getting MTMount siumlator telemetry

In [ ]:
#Check if Rotator telemetry is getting into the EFD
#Add telemetry as needed
#For the "setCompensateMode" of the Hexapods the "angle" of the rotator is needed
rot_sim_Rotation_post_stop_1= await rot_sim.tel_rotation.aget(timeout=10.)
print("Simulated mount elevation", rot_sim_Rotation_post_stop_1.actualPosition)
print("Simulated mount elevation", rot_sim_Rotation_post_stop_1.timestamp)
await asyncio.sleep(10)

In [ ]:
#Check if Rotator telemetry has changed in the EFD
#Add telemetry as needed
rot_sim_Rotation_post_stop_2= await rot_sim.tel_rotation.aget(timeout=10.)
print("Simulated rotator angle", rot_sim_rotation_post_stop_2.timestamp)

In [ ]:
#Check if the input to the EFD has stopped
diff_time =  rot_sim_Rotation_post_stop_1.timestamp - rot_sim_Rotation_post_stop_2.timestamp
print (diff_time)

if (rot_sim_Rotataion_post_stop_1.timestamp -rot_sim_Rotation_post_stop_2.timestamp != 0):
    print("Rotation is receiving telemetry again")
else:
    raise Exception("Rotation is still does not receive telemetry!")